# Biological Data Project

Group members:

- Alberto Calabrese

- Marlon Helbing

- Lorenzo Baietti

"A protein domain is a conserved part of a given protein sequence and tertiary structure that can evolve, function, and exist independently of the rest of the protein chain. Each domain forms a compact three-dimensional structure and often can be independently stable and folded." (Wikipedia).

The project is about the characterization of a single domain. Each group is provided with a representative domain sequence and the corresponding Pfam identifier (see table below). The objective of the project is to build a sequence model starting from the assigned sequence and to provide a functional characterization of the entire domain family (homologous proteins).

## Input
A representative sequence of the domain family. Columns are: group, UniProt accession, organism, Pfam identifier, Pfam name, domain position in the corresponding UniProt protein, domain sequence.

```
UniProt : P54315 
PfamID : PF00151 
Domain Position : 18-353 
Organism : Homo sapiens (Human) 
Pfam Name : Lipase/vitellogenin 
Domain Sequence : KEVCYEDLGCFSDTEPWGGTAIRPLKILPWSPEKIGTRFLLYTNENPNNFQILLLSDPSTIEASNFQMDRKTRFIIHGFIDKGDESWVTDMCKKLFEVEEVNCICVDWKKGSQATYTQAANNVRVVGAQVAQMLDILLTEYSYPPSKVHLIGHSLGAHVAGEAGSKTPGLSRITGLDPVEASFESTPEEVRLDPSDADFVDVIHTDAAPLIPFLGFGTNQQMGHLDFFPNGGESMPGCKKNALSQIVDLDGIWAGTRDFVACNHLRSYKYYLESILNPDGFAAYPCTSYKSFESDKCFPCPDQGCPQMGHYADKFAGRTSEEQQKFFLNTGEASNF
```

## Domain model definition
The objective of the first part of the project is to build a PSSM and HMM model representing the assigned domain. The two models will be generated starting from the assigned input sequence. The accuracy of the models will be evaluated against Pfam annotations as provided in the SwissProt database.

Model Building 
1. Retrieve homologous proteins starting from your input sequence performing a BLAST search
against UniProt or UniRef50 or UniRef90, or any other database

- We use https://www.uniprot.org/blast 
    - use the Domain Sequence as Input
    Parameters : 
    - against UniProtKB
    - e-value thresh : 0.0001
    - 1000 hits

- results in 1000 hits 
- do ID matching to UniProtKB (we need to do that to download the .fasta file)
-  results in 'UNIPROTKB_INITIAL_ORIGINAL.FASTA' 

2. Generate a multiple sequence alignment (MSA) starting from retrieved hits using T-coffee or
ClustalOmega or MUSCLE
    - We use https://www.ebi.ac.uk/jdispatcher/msa/clustalo
    Parameters :
    - Output Format : FASTA

- results in ClustalOmegaUniPortAlignment_ORIGINAL.fasta

3. If necessary, edit the MSA with JalView (or with your custom script or CD-HIT) to remove not
conserved positions (columns) and/or redundant information (rows)
    - We first used JalView at a 100% threshold to check for redundant rows

- results in ClustalOmegaUniPortAlignment.fasta
    - With this we remove down to 155 sequences (check again if correct number)
    - Then we utilize 'conservation.py' to remove columns based on gap frequency (right now only that, lets check if the model is good enough and then we can clean up the code)
    - Right now gap_threshold at 0.90, which removes around 70% of the initial columns (again, need to check actual numbers) ; we should experiment with conservation_threshold and name it in the paper, I think it adds a nice touch that he sees we tried it

- results in trimmed_alignment.fasta



In [1]:
from Bio import AlignIO
from collections import Counter
import pandas as pd
from scipy.stats import entropy
import math
from Bio.Align import MultipleSeqAlignment
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio import SeqIO
import sys

In [ ]:
class ConservationAnalyzer:
    def __init__(self, alignment_file):
        """
        Initialize with an alignment file
            alignment_file (str): Path to the alignment file
        """
        self.alignment = AlignIO.read(alignment_file, 'fasta')
        self.num_sequences = len(self.alignment)
        self.alignment_length = self.alignment.get_alignment_length()
        
    def get_column(self, pos):
        """Extract a column from the alignment"""
        return [record.seq[pos] for record in self.alignment]
    
    def calculate_gap_frequency(self, pos):
        """Calculate frequency of gaps in a column"""
        column = self.get_column(pos)
        return column.count('-') / len(column)
    
    def calculate_amino_acid_frequencies(self, pos):
        """Calculate frequencies of each amino acid in a column"""
        column = self.get_column(pos)
        total = len(column) - column.count('-')  # Don't count gaps, such that when we calculate conservation scores the gaps don't mess it up 
        if total == 0:
            return {}
        
        counts = Counter(aa for aa in column if aa != '-')
        return {aa: count/total for aa, count in counts.items()}
    
    def calculate_conservation_score(self, pos):
        """
        Calculate conservation score based on frequency of most common amino acid
        Ignores gaps in calculation
        """
        freqs = self.calculate_amino_acid_frequencies(pos)
        if not freqs:
            return 0
        return max(freqs.values())
    
    def calculate_entropy(self, pos):
        """
        Calculate Shannon entropy for a column
        Lower entropy means higher conservation
        """
        freqs = self.calculate_amino_acid_frequencies(pos)
        if not freqs:
            return float('inf')  
        
        return -sum(p * math.log2(p) for p in freqs.values())
    
    def get_amino_acid_groups(self):
        """Define groups of similar amino acids 
           Based on : https://en.wikipedia.org/wiki/Conservative_replacement#:~:text=There%20are%2020%20naturally%20occurring,both%20small%2C%20negatively%20charged%20residues.
        """
        return {
            'aliphatic': set('GAVLI'),
            'hydroxyl': set('SCUTM'),
            'cyclic': set('P'),
            'aromatic': set('FYW'),
            'basic': set('HKR'),
            'acidic': set('DENQ')
        }
    
    def calculate_group_conservation(self, pos):
        """
        Calculate conservation considering amino acid groups
        Basically the same as calculate_conversation_score, just that it calculates based on the groups, not single amino acids !
        """
        column = self.get_column(pos)
        groups = self.get_amino_acid_groups()
        
        # Assign each amino acid to its group
        aa_to_group = {}
        for group_name, aas in groups.items():
            for aa in aas:
                aa_to_group[aa] = group_name
        
        # Count group occurrences
        group_counts = Counter(aa_to_group.get(aa, 'other') 
                             for aa in column if aa != '-')
        
        if not group_counts:
            return 0
            
        return max(group_counts.values()) / sum(group_counts.values())





    # TODO : I took very strict values now such that the number of residues per sequence is below 100 (right now we have length 77) ; the PSSM creation with 
    # much higher length did not work, but maybe we should write an email and ask ; nevertheless, we can first try some evaluation based on that PSSM and see our scores

    # TODO : diff gap_thresh/conservation threshold for different number of columns in output (OPTIMIZE)
    def analyze_columns(self, gap_threshold=0.90, conservation_threshold=0.9):
        """
        Analyze all columns and return comprehensive metrics
        Returns DataFrame with various conservation metrics for each position
        """
        data = []
        
        for i in range(self.alignment_length):
            gap_freq = self.calculate_gap_frequency(i)
            cons_score = self.calculate_conservation_score(i)
            info_content = self.calculate_entropy(i)
            group_cons = self.calculate_group_conservation(i)
            
            data.append({
                'position': i + 1,
                'gap_frequency': gap_freq,
                'single_conservation': cons_score,
                'entropy': info_content,
                'group_conservation': group_cons,
                # Here we should look possibly for better ideas
                # Check gap frequency not too high (i.e. not nearly all elements in the columns gaps (-))
                # Check that the group conservation is high enough (i.e. the amino acids are not too different
                # ; right now we do with groups and not single amino acid sequence since I'd say the groups
                # are more representative (if we do single amino acids, we'd delete more stuff))
                'suggested_remove': (gap_freq > gap_threshold) #or       
                                 #  group_cons < conservation_threshold) # TODO : OPTIMIZE WHEN TO REMOVE
            })
        
        return pd.DataFrame(data)


def remove_columns_from_alignment(input_file, output_file, columns_to_remove, format="fasta"):
    """
    Remove specified columns from a multiple sequence alignment and save to new file
    
    Args:
        input_file (str): Path to input alignment file
        output_file (str): Path where to save trimmed alignment
        columns_to_remove (list): List of column indices to remove (0-based)
        format (str): File format (default: "fasta")
    """
    # Read the alignment
    alignment = AlignIO.read(input_file, format)
    
    # Sort columns to remove in descending order
    # (so removing them doesn't affect the indices of remaining columns)
    columns_to_remove = sorted(columns_to_remove, reverse=True)
    
    # Create new alignment records
    new_records = []
    
    # Process each sequence
    for record in alignment:
        # Convert sequence to list for easier manipulation
        seq_list = list(record.seq)
        
        # Remove specified columns
        for col in columns_to_remove:
            del seq_list[col]
        
        # Create new sequence record
        new_seq = Seq(''.join(seq_list)) # Join the list element to a string again (i.e. after removal of amino acids out of sequence represented as list, turn into one string again) and turn into Seq object
        new_record = SeqRecord(new_seq,
                            id=record.id,
                            name=record.name,
                            description=record.description)
        new_records.append(new_record)
    
    # Create new alignment
    # TODO : Maybe we have to add some variables here (i.e. how to do the MSA)!
    new_alignment = MultipleSeqAlignment(new_records)
    
    # Write to file
    AlignIO.write(new_alignment, output_file, format)
    
    return new_alignment

In [ ]:
if __name__ == "__main__":
    # Initialize analyzer 
    analyzer = ConservationAnalyzer("ClustalOmegaUniProtAlignment.fasta")
    
    # Get comprehensive analysis
    analysis = analyzer.analyze_columns(gap_threshold=0.90)
   # analysis_2 = analyzer.analyze_rows()
    
    # Print summary statistics
    print("\nAlignment Summary:")
    print(f"Number of sequences: {analyzer.num_sequences}")
    print(f"Alignment length: {analyzer.alignment_length}")


    # Print number of True/False
    counts = analysis['suggested_remove'].value_counts()

    counts_true = counts[True]  # To be removed
    counts_false = counts[False] # To be kept

    print(f"With the current removal tactic, we would remove {(counts_true / (counts_true + counts_false)):.2f} percent of columns ; we keep {counts_false} of {counts_false + counts_true} columns")
    

    # Save detailed analysis to CSV
    analysis.to_csv("conservation_analysis.csv", index=False)


    # Get indices of columns marked for removal
    columns_to_remove = analysis[analysis['suggested_remove']]['position'].values.tolist()
    # Convert to 0-based indices (if positions were 1-based)
    columns_to_remove = [x-1 for x in columns_to_remove]
    
    # Remove columns and save new alignment
    new_alignment = remove_columns_from_alignment(
        "ClustalOmegaUniProtAlignment.fasta",
        "trimmed_alignment.fasta",
        columns_to_remove
    )


        


    print(f"Original alignment length: {analyzer.alignment_length}")
    print(f"Number of columns removed: {len(columns_to_remove)}")
    print(f"New alignment length: {new_alignment.get_alignment_length()}")

4. Build a PSSM model starting from the MSA

- First direct in the folder where ncbi-blast-2.16.0+ was installed

- Then use this terminal command : 

ncbi-blast-2.16.0+/bin/psiblast -subject data/protein_family/trimmed_alignment.fasta -in_msa data/protein_family/trimmed_alignment.fasta -out_ascii_pssm data/protein_family/trimmed_alignment.pssm_ascii -out_pssm data/protein_family/trimmed_alignment.pssm

    - Note that the trimmed_alignment.fasta needs to be in the described folder (data/protein_family/)


5. Build a HMM model starting from the MSA

- First direct in the folder where hmmer-3.4 was installed

- Then use this terminal command :

hmmer-3.4/src/hmmbuild data/protein_family/trimmed_alignment.hmm data/protein_family/trimmed_alignment.fasta

    - Note that the trimmed_alignment.fasta needs to be in the described folder (data/protein_family/)



## Models evaluation
1. Generate predictions. Run HMM-SEARCH and PSI-BLAST with your models against
SwissProt.

    - Collect the list of retrieved hits

    - Collect matching positions of your models in the retrieved hits

PSI-BLAST :
-	When working on MAC : First have to change settings so we have access to use psiblast & makeblastdb (in terminal, go to folder where psiblast/makeblastdb located and run this)
    xattr -c psiblast
    chmod +x psiblast
    xattr -c makeblastdb
    chmod +x makeblastdb

- Then use this command to create a "formatted swissprot database" (we need to check what that exactly means)
      
      ./ncbi-blast-2.16.0+/bin/makeblastdb -in uniprot_sprot.fasta -dbtype prot -out swissprot

        - Notice that uniprot_sprot.fasta needs to be installed and be located in the current folder that we are in (in the terminal)


- Finally, the actual predictions can be obtained with the following command

        ./ncbi-blast-2.16.0+/bin/psiblast -in_pssm trimmed_alignment.pssm \
            -db swissprot \
            -out psiblast_search_output.txt \
            -outfmt "6 qseqid sseqid qstart qend sstart send pident evalue" \

    - where 

            qseqid: Query sequence identifier (your domain)
            sseqid: Subject sequence identifier (matched protein)
            qstart: Start position in your query domain
            qend: End position in your query domain
            sstart: Start position in the matched sequence
            send: End position in the matched sequence
            pident: Percentage of identical matches
            evalue: Expectation value (statistical significance)
    

HMMER :

- Simply use 

    ./hmmer-3.4/src/hmmsearch trimmed_alignment.hmm uniprot_sprot.fasta > hmmsearch_output.txt


- To make both of the output files more accessible, we parsed them by writing two scripts 
    - in that way we create two .csv files that are easier to be compared, also directly by eye
    - these .csv files then contained the matching positions of our two models in the retrieved hits

Returns psiblast_parsed.csv
Returns hmmsearch_output.csv

In [ ]:
import csv
import re 

In [ ]:
"""FOR PSIBLAST"""
def parse_psiblast_output(input_file):
    results = []
    
    with open(input_file, 'r') as f:
        for line in f:
            # Skip empty lines
            if not line.strip():
                continue
                
            # Split the line by tabs or multiple spaces
            parts = re.split(r'\s+', line.strip())
            
            if len(parts) >= 8:  # Make sure we have all required fields
                query_id = parts[0]
                subject_id = parts[1]
                
                # Extract UniProt ID and organism from subject_id
                # Format is usually sp|UniprotID|Name
                subject_parts = subject_id.split('|')
                if len(subject_parts) >= 2:
                    uniprot_id = subject_parts[1]
                    
                    # Create result dictionary
                    result = {
                        'protein_name': subject_id,
                        'uniprot_id': uniprot_id,
                        'organism': 'N/A',  # PSIBLAST output doesn't include organism
                        'domain_start': int(parts[4]),  # sstart
                        'domain_end': int(parts[5]),    # send
                        'domain_length': int(parts[5]) - int(parts[4]) + 1,
                        'E-value': float(parts[7])  
                    }
                    results.append(result)
    
    return results

def write_csv(results, output_file):
    if not results:
        return
    # Notice that we skip the start & end positions in the query domain (i.e. the PSSM here), as we are only interested in where we found matches in the sequence of SwissProt we looked through
    fieldnames = ['protein_name', 'uniprot_id', 'organism', 'domain_start', 
                 'domain_end', 'domain_length', 'E-value']
    
    with open(output_file, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(results)


input_file = 'psiblast_search_output.txt'
output_file = 'psiblast_parsed.csv'

results = parse_psiblast_output(input_file)
write_csv(results, output_file)
print(f"Processed {len(results)} PSIBLAST matches")

In [ ]:
"""FOR HMM"""
# File paths
input_file_path = "hmmsearch_output.txt"
output_file_path = "hmmsearch_output.csv"

# Initialize storage for parsed data
parsed_data = []

# Regular expressions to capture key information
header_regex = r">> ([^\s]+)"
domain_regex = r"\s+(\d+) [!?]\s+[\d\.]+\s+[\d\.]+\s+[\de\.\+\-]+\s+([\de\.\+\-]+)\s+\d+\s+\d+\s+(?:\[\.|\.\.)+\s+(\d+)\s+(\d+)"

with open(input_file_path, "r") as infile:
    current_protein = None

    for line in infile:
        # Match protein header line
        header_match = re.match(header_regex, line)
        if header_match:
            # If we already captured a protein, save its data
            if current_protein:
                parsed_data.append(current_protein)

            # Start a new protein record
            protein_id = header_match.groups()[0]
            current_protein = {
                "protein_name": protein_id.split("|")[2],
                "uniprot_id": protein_id.split("|")[1],
                "domains": []
            }

        # Match domain annotation (including both `!` and `?` lines)
        domain_match = re.match(domain_regex, line)
        if domain_match and current_protein:
            _, score, start, end = domain_match.groups()
            start, end, score = int(start), int(end), float(score)
            length = end - start + 1
            current_protein["domains"].append((score, start, end, length))

    # Handle the last protein record
    if current_protein:
        parsed_data.append(current_protein)

# Prepare fieldnames dynamically
fieldnames = ["protein_name", "uniprot_id"]
max_domains = max(len(protein["domains"]) for protein in parsed_data)
for i in range(1, max_domains + 1):
    if i == 1:
        fieldnames.extend([
            f"E-value", f"domain_start", f"domain_end", f"domain_length"
        ])
    else:
        fieldnames.extend([
        f"domain_{i}_E-value", f"domain_{i}_start", f"domain_{i}_end", f"domain_{i}_length"
    ])

# Write to CSV
with open(output_file_path, "w", newline="") as outfile:
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    writer.writeheader()
    for protein in parsed_data:
        row = {
            "protein_name": protein["protein_name"],
            "uniprot_id": protein["uniprot_id"]
        }
        for i, domain in enumerate(protein["domains"], start=1):
            if i == 1:
                row[f"E-value"] = domain[0]
                row[f"domain_start"] = domain[1]
                row[f"domain_end"] = domain[2]
                row[f"domain_length"] = domain[3] 
            else:
                row[f"domain_{i}_E-value"] = domain[0]
                row[f"domain_{i}_start"] = domain[1]
                row[f"domain_{i}_end"] = domain[2]
                row[f"domain_{i}_length"] = domain[3]
        writer.writerow(row)

print(f"CSV file generated: {output_file_path}")


2. Define your ground truth. Find all proteins in SwissProt annotated (and not annotated) with the assigned Pfam domain

    - Collect the list of proteins matching the assigned Pfam domain

    - Collect matching positions of the Pfam domain in the retrieved sequences. Domain positions are available here (large tsv file) or using the InterPro API or align the Pfam domain yourself against SwissProt (HMMSEARCH)

        - For this, we decided to use the InterPro API : 

        https://www.ebi.ac.uk/interpro/api/protein/reviewed/entry/pfam/PF00151/

        which contains the reviewed proteins that match our assigned PFAM domain, PF00151

        - to effectively scrape this website, we wrote a parser

    Returns pfam_domain_positions.json 



In [ ]:
import json
import requests
import time
from typing import Dict, List, Optional
from collections import defaultdict

In [ ]:
class InterProAPIFetcher:
    def __init__(self, base_url: str):
        self.base_url = base_url
        self.processed_count = 0
        self.all_results = []
        self.seen_accessions = set()  # Track unique protein accessions
        self.duplicate_count = 0

    def fetch_page(self, url: str) -> Optional[Dict]:
        max_retries = 3
        retry_delay = 2
        
        for attempt in range(max_retries):
            try:
                response = requests.get(url)
                response.raise_for_status()
                return response.json()
            except requests.exceptions.RequestException as e:
                print(f"Attempt {attempt + 1} failed: {str(e)}")
                if attempt < max_retries - 1:
                    print(f"Waiting {retry_delay} seconds before retrying...")
                    time.sleep(retry_delay)
                    retry_delay *= 2
                else:
                    print("Max retries reached. Moving on...")
                    return None

    def fetch_all_pages(self) -> List[Dict]:
        next_url = self.base_url
        total_count = None
        page_number = 1
        
        while next_url:
            print(f"\nFetching page {page_number}...")
            print(f"URL: {next_url}")
            
            page_data = self.fetch_page(next_url)
            
            if page_data is None:
                print("Failed to fetch page. Stopping pagination.")
                break
            
            if total_count is None:
                total_count = page_data['count']
                print(f"API reports total count: {total_count}")
            
            # Check for duplicates in this page
            new_proteins = []
            page_duplicates = 0
            
            for protein in page_data['results']:
                accession = protein['metadata']['accession']
                if accession in self.seen_accessions:
                    page_duplicates += 1
                    self.duplicate_count += 1
                else:
                    self.seen_accessions.add(accession)
                    new_proteins.append(protein)
            
            print(f"Page {page_number} stats:")
            print(f"- Proteins in response: {len(page_data['results'])}")
            print(f"- New unique proteins: {len(new_proteins)}")
            print(f"- Duplicates found: {page_duplicates}")
            
            self.all_results.extend(new_proteins)
            self.processed_count = len(self.all_results)
            
            next_url = page_data.get('next')
            page_number += 1
            
            time.sleep(1)
        
        print(f"\nFinal Statistics:")
        print(f"Total unique proteins: {len(self.all_results)}")
        print(f"Total duplicates found: {self.duplicate_count}")
        print(f"Total processed entries: {self.processed_count + self.duplicate_count}")
        
        return self.all_results

    def save_results(self, filename: str):
        output_data = {
            'count': len(self.all_results),
            'results': self.all_results
        }
        
        with open(filename, 'w') as f:
            json.dump(output_data, f, indent=2)
        print(f"\nResults saved to {filename}")
        print(f"File contains {len(self.all_results)} unique proteins")

def main():
    base_url = "https://www.ebi.ac.uk/interpro/api/protein/reviewed/entry/pfam/PF00151/"
    
    fetcher = InterProAPIFetcher(base_url)
    fetcher.fetch_all_pages()
    fetcher.save_results('pfam_domain_positions.json')

if __name__ == "__main__":
    main()

- After the successful parsing, we also turn the .json file into .csv for further processing (and again, easier interpretation by eye)

For this we wrote another script 

Returns pfam_domain_positions.csv

In [ ]:
import json

In [ ]:
def extract_pfam_info(json_file):
    # Read the JSON file
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    # List to store the extracted information
    pfam_matches = []
    
    # Iterate through each result in the JSON data
    for result in data['results']:
        # Extract protein metadata
        protein_info = {
            'protein_name': result['metadata']['name'],
            'uniprot_id': result['metadata']['accession'],
            'organism': result['metadata']['source_organism']['scientificName']
        }
        
        # Extract PFAM domain information
        # We know there's only one entry because we queried for a specific PFAM domain
        pfam_entry = result['entries'][0]
        
        # Get the domain fragments (start and end positions)
        for location in pfam_entry['entry_protein_locations']:
            for fragment in location['fragments']:
                domain_info = {
                    **protein_info,  # Include all protein information
                    'domain_start': fragment['start'],
                    'domain_end': fragment['end'],
                    'domain_length': fragment['end'] - fragment['start'] + 1,
                    'protein_length': pfam_entry['protein_length'],
                    'score': location['score']
                }
                pfam_matches.append(domain_info)
    
    return pfam_matches

# Use the function to extract information
json_file = 'pfam_domain_positions.json'
matches = extract_pfam_info(json_file)

# Print the results in a formatted way
print("\nPFAM Domain Matches:")
print("-" * 80)
for match in matches:
    print(f"Protein: {match['protein_name']} ({match['uniprot_id']})")
    print(f"Organism: {match['organism']}")
    print(f"Domain position: {match['domain_start']}-{match['domain_end']} "
          f"(length: {match['domain_length']} aa)")
    print(f"Total protein length: {match['protein_length']} aa")
    print(f"Score: {match['score']}")
    print("-" * 80)

# Optional: Save to a more structured format like CSV for further analysis
import pandas as pd

# Convert the matches to a DataFrame
df = pd.DataFrame(matches)

# Save to CSV
df.to_csv('pfam_domain_positions.csv', index=False)
print("\nResults have been saved to 'pfam_domain_positions.csv'")

- Finally, we now have 3 similar .csv files , 1 depicting the Ground Truth and 2 depicting the predictions from both the PSSM and HMM model



3. Compare your model with the assigned Pfam. Calculate the precision, recall, F-score, balanced accuracy, MCC

    - Comparison at the protein level. Measure the ability of your model to retrieve the same proteins matched by Pfam

    - Comparison at the residue level. Measure the ability of your model to match the same position matched by Pfam


- To this extent , we created a script for evaluation purposes
- Here also explain the code more (e.g. the one hot encoding for residue levels, that for the HMM we always just took the first hit with the lowest e-value and didn't look at the other hits because they never were significant etc.)



In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef

In [ ]:
"""Based on the .csv files that closely resemble each other, do the calculations"""


def create_residue_vectors(pred_df, pfam_df, protein_id):
    """
    For a single protein (based on its protein_id), create residue vectors
    """

    # Note that since the ground truth (i.e. the PFAM domains) are all just a single sequence hit, we only include the strongest
    # (lowest e-value) hit from the HMM search (also we noticed that basically all 2nd, 3rd domain hits have much higher e-values)
 
    # Get domain positions from both predictions
    pred_matches = pred_df[pred_df['uniprot_id'] == protein_id]
    pfam_matches = pfam_df[pfam_df['uniprot_id'] == protein_id]
    
    # Check if for the protein at hand, we even find it in both of the .csv's 
    if len(pred_matches) == 0 or len(pfam_matches) == 0:
        return None
    

    assert (len(pred_df == 1) , "not length 1 ")
    # Get the max of the max lengths for each hit for the current protein
    # Notice that we can have multiple hits per protein (i.e. multiple alignments that were found)
    # So we have to account for all of them
    max_length = max(
        pred_matches['domain_end'].max(),
        pfam_matches['domain_end'].max()
    )
    
    # With that, we can create vectors of the same size
    # Create binary vectors for each position
    true_positions = np.zeros(int(max_length))
    pred_positions = np.zeros(int(max_length))
    

    # Now, iterate through all the found alignments (for the current protein)

    # Fill in Pfam (true) positions
    for _, row in pfam_matches.iterrows():
        start = row['domain_start'] - 1  # Convert to 0-based indexing
        end = row['domain_end']
        true_positions[start:end] = 1
        
    # Fill in PSSM/HMM (predicted) positions
    for _, row in pred_matches.iterrows():
        start = int(row['domain_start'] - 1)  # Convert to 0-based indexing
        end = int(row['domain_end'])
        pred_positions[start:end] = 1
    
    return true_positions, pred_positions




def evaluate_model(psiblast_file, hmm_file, pfam_file, only_found=False, e_threshold = 0.0001):
    """
    Evaluate PSIBLAST model performance against Pfam annotations
    
    Parameters:
    - psiblast_file: Path to PSIBLAST results CSV
    - hmm_file : Path to HMM results CSV
    - pfam_file: Path to Pfam ground truth CSV
    - only_found: If True, only evaluate proteins found by PSIBLAST
    """
    # Step 1: Load both CSV files
    psiblast_df = pd.read_csv(psiblast_file)
    hmm_df = pd.read_csv(hmm_file)
    pfam_df = pd.read_csv(pfam_file)

    # HMM finds a lot of hits, a lot with extremely high e-values : Take only the ones that are above some threshold (score for now, later e-value)
    # We filter based on the first domain hit (TODO : i.e. the best one ? check if first found domain always strongest)
    filtered_hmm_proteins = hmm_df[hmm_df['E-value'] <= e_threshold]['uniprot_id']
    
    # Step 2: Get unique list of proteins from both files
    psiblast_proteins = set(psiblast_df['uniprot_id'])
    hmm_proteins = set(filtered_hmm_proteins)
    pfam_proteins = set(pfam_df['uniprot_id'])

    
    if only_found:
        # Only consider proteins that PSIBLAST/HMM found
        all_proteins_psiblast = psiblast_proteins
        all_proteins_hmm = hmm_proteins
        print("\nEvaluating only PSIBLAST-found proteins:")
    else:
        # Consider all proteins from both sets
        all_proteins_psiblast = psiblast_proteins.union(pfam_proteins)
        all_proteins_hmm = hmm_proteins.union(pfam_proteins)
        print("\nEvaluating all proteins:")
    
    print(f"Number of proteins predicted by PSIBLAST: {len(psiblast_proteins)}")
    print(f"Number of proteins predicted by HMM: {len(hmm_proteins)}")
    print(f"Number of proteins in Pfam ground truth: {len(pfam_proteins)}")
    print(f"Number of proteins being evaluated for PSIBLAST: {len(all_proteins_psiblast)}")
    print(f"Number of proteins being evaluated for HMM: {len(all_proteins_hmm)}")
    

    print("\n=== Protein-Level Evaluation ===")
    # Step 3: Create binary vectors for true and predicted labels
    y_true_psiblast = []  # Ground truth from Pfam
    y_pred_psiblast = []  # Predictions from PSIBLAST
    y_true_hmm = []
    y_pred_hmm = []


    
    for protein in all_proteins_psiblast:
        y_true_psiblast.append(1 if protein in pfam_proteins else 0)
        y_pred_psiblast.append(1 if protein in psiblast_proteins else 0)


    for protein in all_proteins_hmm:
        y_true_hmm.append(1 if protein in pfam_proteins else 0)
        y_pred_hmm.append(1 if protein in hmm_proteins else 0)

    # So we have something like
    # y_true_psiblast 0 0 1 0 1 ...
    # y_pred_psiblast 0 1 1 0 1 ...
    
    # Step 4: Calculate performance metrics
    protein_results_psiblast = {
        'Precision': precision_score(y_true_psiblast, y_pred_psiblast),
        'Recall': recall_score(y_true_psiblast, y_pred_psiblast),
        'F-score': f1_score(y_true_psiblast, y_pred_psiblast),
        'Balanced Accuracy': balanced_accuracy_score(y_true_psiblast, y_pred_psiblast),
        'MCC': matthews_corrcoef(y_true_psiblast, y_pred_psiblast)
    }


    protein_results_hmm = {
        'Precision': precision_score(y_true_hmm, y_pred_hmm),
        'Recall': recall_score(y_true_hmm, y_pred_hmm),
        'F-score': f1_score(y_true_hmm, y_pred_hmm),
        'Balanced Accuracy': balanced_accuracy_score(y_true_hmm, y_pred_hmm),
        'MCC': matthews_corrcoef(y_true_hmm, y_pred_hmm)
    }


    # Step 5: Calculate confusion matrix components
    tp_psiblast = sum(1 for t, p in zip(y_true_psiblast, y_pred_psiblast) if t == 1 and p == 1)
    fp_psiblast = sum(1 for t, p in zip(y_true_psiblast, y_pred_psiblast) if t == 0 and p == 1)
    fn_psiblast = sum(1 for t, p in zip(y_true_psiblast, y_pred_psiblast) if t == 1 and p == 0)
    tn_psiblast = sum(1 for t, p in zip(y_true_psiblast, y_pred_psiblast) if t == 0 and p == 0)


    tp_hmm = sum(1 for t, p in zip(y_true_hmm, y_pred_hmm) if t == 1 and p == 1)
    fp_hmm = sum(1 for t, p in zip(y_true_hmm, y_pred_hmm) if t == 0 and p == 1)
    fn_hmm = sum(1 for t, p in zip(y_true_hmm, y_pred_hmm) if t == 1 and p == 0)
    tn_hmm = sum(1 for t, p in zip(y_true_hmm, y_pred_hmm) if t == 0 and p == 0)
    
    # Print detailed results
    print("\nProtein-Level Confusion Matrix PSIBLAST:")
    print(f"True Positives: {tp_psiblast}")
    print(f"False Positives: {fp_psiblast}")
    print(f"False Negatives: {fn_psiblast}")
    print(f"True Negatives: {tn_psiblast}")



    print("\nProtein-Level Confusion Matrix HMM:")
    print(f"True Positives: {tp_hmm}")
    print(f"False Positives: {fp_hmm}")
    print(f"False Negatives: {fn_hmm}")
    print(f"True Negatives: {tn_hmm}")


    print("\nProtein-Level Metrics PSIBLAST:")
    for metric, value in protein_results_psiblast.items():
        print(f"{metric}: {value:.4f}")


    print("\nProtein-Level Metrics HMM:")
    for metric, value in protein_results_hmm.items():
        print(f"{metric}: {value:.4f}")



    # Residue-level evaluation
   
    print("\n=== Residue-Level Evaluation ===")
    # Only evaluate residues for proteins found in both sets 
    common_proteins_psiblast = psiblast_proteins.intersection(pfam_proteins)
    common_proteins_hmm = hmm_proteins.intersection(pfam_proteins)
    print(f"Number of proteins for residue-level evaluation PSIBLAST: {len(common_proteins_psiblast)}")
    print(f"Number of proteins for residue-level evaluation HMM: {len(common_proteins_hmm)}")
    
    # Collect all residue-level predictions
    all_true_residues_psiblast = []
    all_pred_residues_psiblast = []

    all_true_residues_hmm = []
    all_pred_residues_hmm = []

    
    for protein in common_proteins_psiblast:
        result = create_residue_vectors(psiblast_df, pfam_df, protein)
        if result is not None:
            true_pos, pred_pos = result
            all_true_residues_psiblast.extend(true_pos)
            all_pred_residues_psiblast.extend(pred_pos)


    for protein in common_proteins_hmm:
        result = create_residue_vectors(hmm_df, pfam_df, protein)
        if result is not None:
            true_pos, pred_pos = result
            all_true_residues_hmm.extend(true_pos)
            all_pred_residues_hmm.extend(pred_pos)
    
    # Calculate residue-level metrics
    residue_results_psiblast = {
        'Precision': precision_score(all_true_residues_psiblast, all_pred_residues_psiblast),
        'Recall': recall_score(all_true_residues_psiblast, all_pred_residues_psiblast),
        'F-score': f1_score(all_true_residues_psiblast, all_pred_residues_psiblast),
        'Balanced Accuracy': balanced_accuracy_score(all_true_residues_psiblast, all_pred_residues_psiblast),
        'MCC': matthews_corrcoef(all_true_residues_psiblast, all_pred_residues_psiblast)
    }
    
    # Calculate residue-level confusion matrix
    tp = sum(1 for t, p in zip(all_true_residues_psiblast, all_pred_residues_psiblast) if t == 1 and p == 1)
    fp = sum(1 for t, p in zip(all_true_residues_psiblast, all_pred_residues_psiblast) if t == 0 and p == 1)
    fn = sum(1 for t, p in zip(all_true_residues_psiblast, all_pred_residues_psiblast) if t == 1 and p == 0)
    tn = sum(1 for t, p in zip(all_true_residues_psiblast, all_pred_residues_psiblast) if t == 0 and p == 0)
    
    print("\nResidue-Level Confusion Matrix PSIBLAST:")
    print(f"True Positives: {tp}")
    print(f"False Positives: {fp}")
    print(f"False Negatives: {fn}")
    print(f"True Negatives: {tn}")
    
    print("\nResidue-Level Metrics PSIBLAST:")
    for metric, value in residue_results_psiblast.items():
        print(f"{metric}: {value:.4f}")



    # Calculate residue-level metrics
    residue_results_hmm = {
        'Precision': precision_score(all_true_residues_hmm, all_pred_residues_hmm),
        'Recall': recall_score(all_true_residues_hmm, all_pred_residues_hmm),
        'F-score': f1_score(all_true_residues_hmm, all_pred_residues_hmm),
        'Balanced Accuracy': balanced_accuracy_score(all_true_residues_hmm, all_pred_residues_hmm),
        'MCC': matthews_corrcoef(all_true_residues_hmm, all_pred_residues_hmm)
    }
    
    # Calculate residue-level confusion matrix
    tp = sum(1 for t, p in zip(all_true_residues_hmm, all_pred_residues_hmm) if t == 1 and p == 1)
    fp = sum(1 for t, p in zip(all_true_residues_hmm, all_pred_residues_hmm) if t == 0 and p == 1)
    fn = sum(1 for t, p in zip(all_true_residues_hmm, all_pred_residues_hmm) if t == 1 and p == 0)
    tn = sum(1 for t, p in zip(all_true_residues_hmm, all_pred_residues_hmm) if t == 0 and p == 0)


    print("\nResidue-Level Confusion Matrix HMM:")
    print(f"True Positives: {tp}")
    print(f"False Positives: {fp}")
    print(f"False Negatives: {fn}")
    print(f"True Negatives: {tn}")
    
    print("\nResidue-Level Metrics HMM:")
    for metric, value in residue_results_hmm.items():
        print(f"{metric}: {value:.4f}")
    



psiblast_file = 'psiblast_parsed.csv'
hmm_file = 'hmmsearch_output.csv'
pfam_file = 'pfam_domain_positions.csv'

#print("===============================")
#print("Evaluation on all proteins:")
#results_all = evaluate_model_protein_level(psiblast_file, pfam_file, only_found=False)

print("\n===============================")
print("Evaluation only on found proteins in both PSSM/HMM:")
evaluate_model(psiblast_file,hmm_file, pfam_file, only_found=False, e_threshold= 0.001)

4. Consider refining your models to improve their performance

- Multiple Steps that we took

    - remove redundancy at different thresholds in JalView
    - use other starting database
    - use in first search (Step 1) different e-values for broader searches
    - when removing columns, tweak the two parameters that we had differently to see what gives better results
    - and here then say we found our best model using ....

## Domain family characterization
Once the family model is defined (previous step), you will look at functional (and structural) aspects/properties of the entire protein family. The objective is to provide insights about the main function of the family.

### Taxonomy

1. Collect the taxonomic lineage (tree branch) for each protein of the family_sequences dataset
from UniProt (entity/organism/lineage in the UniProt XML)

2. Plot the taxonomic tree of the family with nodes size proportional to their relative abundance 


In [1]:
import requests
import csv
import pandas as pd
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist
import numpy as np
from typing import List, Dict, Tuple
import time
from tqdm import tqdm
import logging
from Bio import Phylo
import matplotlib.pyplot as plt
import pandas as pd
from io import StringIO
from ete3 import Tree, TreeStyle, NodeStyle, TextFace



In [2]:
# Take a union of the proteins found by HMM (with e-value thresh of 0.001) and PSIBLAST (all 21) to represent our "family_sequences"
e_threshold = 0.001
psiblast_df = pd.read_csv("psiblast_parsed.csv")
hmm_df = pd.read_csv("hmmsearch_output.csv")
filtered_hmm_proteins = hmm_df[hmm_df['E-value'] <= e_threshold]['uniprot_id']
    
psiblast_proteins = set(psiblast_df['uniprot_id'])
hmm_proteins = set(filtered_hmm_proteins)


family_sequences = list(psiblast_proteins.union(hmm_proteins))





In [3]:
# TaxonomyAnalyzer Class for fetching taxonomy information
class TaxonomyAnalyzer:
    def __init__(self, max_retries: int = 3, retry_delay: int = 1):
        self.max_retries = max_retries
        self.retry_delay = retry_delay
        self.uniprot_base_url = "https://rest.uniprot.org/uniprotkb/"

    def fetch_taxonomy_info(self, protein_ids: list, output_file: str):
        taxonomy_data = []

        pbar = tqdm(protein_ids, desc="Fetching taxonomy data")

        for protein_id in pbar:
            pbar.set_description(f"Processing {protein_id}")

            for attempt in range(self.max_retries):
                try:
                    response = requests.get(f"{self.uniprot_base_url}{protein_id}.json")
                    response.raise_for_status()
                    data = response.json()

                    taxonomy = data.get("organism", {})
                    scientific_name = taxonomy.get("scientificName", "N/A")
                    lineage = taxonomy.get("lineage", [])

                    taxonomy_data.append([protein_id, scientific_name, " > ".join(lineage)])
                    break

                except requests.exceptions.RequestException as e:
                    print(f"Error fetching data for {protein_id}: {e}")
                    if attempt == self.max_retries - 1:
                        taxonomy_data.append([protein_id, "Error", ""])
                    else:
                        time.sleep(self.retry_delay)

        taxonomy_df = pd.DataFrame(taxonomy_data, columns=["Protein ID", "Scientific Name", "Lineage"])
        taxonomy_df.to_csv(output_file, index=False)
        return taxonomy_df

# Load protein IDs from files
def load_protein_ids(psiblast_file, hmm_file, e_threshold=0.001):
    psiblast_df = pd.read_csv(psiblast_file)
    hmm_df = pd.read_csv(hmm_file)

    filtered_hmm_proteins = hmm_df[hmm_df['E-value'] <= e_threshold]['uniprot_id']
    psiblast_proteins = set(psiblast_df['uniprot_id'])
    hmm_proteins = set(filtered_hmm_proteins)

    return list(psiblast_proteins.union(hmm_proteins))

# Process taxonomy data
def process_taxonomy(data, correct_column_name):
    taxonomy_dict = {}
    frequency_counts = {}
    
    for _, row in data.iterrows():
        lineage = row[correct_column_name].split(" > ")
        current = taxonomy_dict
        # Track the full path to maintain hierarchy information
        current_path = [] # such that we count occurences of terms in the correct "level" where they appear (i.e. always count just in the "column" of the linage)
        
        for level in lineage:
            current_path.append(level)
            path_key = " > ".join(current_path)
            
            # Count frequencies using the full path as key
            if path_key not in frequency_counts:
                frequency_counts[path_key] = 0
            frequency_counts[path_key] += 1
            
            if level not in current:
                current[level] = {}
            current = current[level]
    
    return taxonomy_dict, frequency_counts

# Create a Newick string for the taxonomy tree
def dict_to_newick(d, parent_abundance=None):
    newick = ""
    for key, sub_dict in d.items():
        size = parent_abundance.get(key, 1) if parent_abundance else 1
        sub_tree = dict_to_newick(sub_dict, parent_abundance)
        newick += f"({sub_tree}){key}:{size}," if sub_tree else f"{key}:{size},"
    return newick.rstrip(",")



# Fetch taxonomy data
def main():
    psiblast_file = "psiblast_parsed.csv"
    hmm_file = "hmmsearch_output.csv"
    protein_ids = load_protein_ids(psiblast_file, hmm_file)

    analyzer = TaxonomyAnalyzer()
    taxonomy_data = analyzer.fetch_taxonomy_info(protein_ids, "taxonomy_info.csv")

    print("Taxonomy file saved to: taxonomy_info.csv")

    # Correct column name
    correct_column_name = "Lineage"  # Use the correct column name

    # Create a nested dictionary of taxonomy
    taxonomy_dict, frequency_counts = process_taxonomy(taxonomy_data, correct_column_name)

    # Count relative abundance (of the different paths ! ; right now we don't really use that)
    abundance_counts = taxonomy_data[correct_column_name].value_counts().to_dict()


    newick_tree = f"({dict_to_newick(taxonomy_dict, abundance_counts)});"

    # Plot using ETE Toolkit
    phylo_tree = Tree(newick_tree, format=1)
    tree_style = TreeStyle()
    tree_style.show_leaf_name = False


    # Adjust node sizes (normalize and refine scaling)
    max_size = 50  # Increase max size for better differentiation
    scaling_factor = 2  # Further refine scaling for visual contrast
    for node in phylo_tree.traverse():
        # Get the full path from root to this node
        path = []
        current = node
        while current:
            if current.name:  # Skip empty names
                path.insert(0, current.name)
            current = current.up
        
        path_key = " > ".join(path)
        count = frequency_counts.get(path_key, 1)
        nstyle = NodeStyle()
        size = abundance_counts.get(node.name, 1)
        nstyle["size"] = min(size * scaling_factor, max_size)  # Scale and cap node size
        node.set_style(nstyle)
        # Add label with name and count
        node.add_face(TextFace(f"{node.name} ({count})", fsize=10), column=0)

    # Improve tree spacing
    tree_style.branch_vertical_margin = 30  # Increase spacing for better visibility

    # Save the tree to a high-resolution PNG file
    output_file = "phylogenetic_tree_freq.png"
    phylo_tree.render(output_file, w=3000, h=2000, tree_style=tree_style)

    print(f"Tree saved to: {output_file}")

if __name__ == "__main__":
    main()


Processing P54315: 100%|██████████| 83/83 [00:17<00:00,  4.86it/s]    


Taxonomy file saved to: taxonomy_info.csv
Tree saved to: phylogenetic_tree_freq.png


### Function

1. Collect GO annotations for each protein of the family_sequences dataset (entity/dbReference type="GO" in the UniProt XML)

2. Calculate the enrichment of each term in the dataset compared to GO annotations available in the SwissProt database (you can download the entire SwissProt XML here). You can use Fisher’ exact test and verify that both two-tails and right-tail P-values (or left-tail depending on how you build the confusion matrix) are close to zero

3. Plot enriched terms in a word cloud 

4. Take into consideration the hierarchical structure of the GO ontology and report most significantly enriched branches, i.e. high level terms

5. Always report the full name of the terms and not only the GO ID

In [1]:
!pip install obonet
!pip install statsmodels
!pip install goatools

In [ ]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
from scipy.stats import fisher_exact
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import random
import obonet
import networkx as nx
from statsmodels.stats.multitest import multipletests
from collections import defaultdict
from goatools import obo_parser



In [ ]:
# Helper function to turn a Protein_ID : [GO_terms] into a GO_term : [Protein_IDs] dictionary

def reverse_protein_go_dict(protein_to_go):
   """
   Convert protein->GO dict to GO->proteins dict.
   """
   go_to_proteins = defaultdict(list)
   for protein, go_terms in protein_to_go.items():
       for go_term in go_terms:
           go_to_proteins[go_term].append(protein)
   return go_to_proteins

In [59]:

# Step 1: Load Protein IDs
# TODO : we basically did this above already for taxonomy task and just here neatly written into a function, so we could maybe just do it once in the whole code later on
def load_protein_ids(psiblast_file, hmm_file, e_threshold=0.001):
    """Load protein IDs from PSI-BLAST and HMM search results."""
    psiblast_df = pd.read_csv(psiblast_file)
    hmm_df = pd.read_csv(hmm_file)
    
    filtered_hmm_proteins = hmm_df[hmm_df['E-value'] <= e_threshold]['uniprot_id']
    psiblast_proteins = set(psiblast_df['uniprot_id'])
    hmm_proteins = set(filtered_hmm_proteins)
    
    return list(psiblast_proteins.union(hmm_proteins))

'''
def fetch_go_annotations(protein_id):
    """
    Fetch and categorize GO annotations for a given protein ID from the UniProt API.
    
    Args:
        protein_id (str): The UniProt ID of the protein
        
    Returns:
        dict: A dictionary containing:
            - Categorized GO terms separated by molecular function, biological process, 
              and cellular component (new format)
    """
    # Define the UniProt API URL for XML data
    url = f"https://rest.uniprot.org/uniprotkb/{protein_id}.xml"

    try:
        # Fetch the XML data from UniProt
        response = requests.get(url)
        response.raise_for_status()
        
        # Initialize our data structures
        go_terms = []  # Original format
        categorized_terms = {
            'molecular_function': [],
            'biological_process': [],
            'cellular_component': []
        }
        
        # Set up namespace for XML parsing
        namespaces = {'ns': 'http://uniprot.org/uniprot'}
        root = ET.fromstring(response.content)
        
        # Find all GO term references in the XML
        for db_ref in root.findall(".//ns:dbReference[@type='GO']", namespaces):
            go_id = db_ref.attrib.get('id')
            term = db_ref.find("ns:property[@type='term']", namespaces)

            go_term = term.get('value')
            
            if go_id and term is not None:
                # Store in original format
                term_value = term.attrib['value']
                
                # Categorize based on prefix
                if term_value.startswith('F:'):
                    categorized_terms['molecular_function'].append({
                        'id': go_id,
                        'term': term_value[2:]  # Remove 'F:' prefix
                    })
                elif term_value.startswith('P:'):
                    categorized_terms['biological_process'].append({
                        'id': go_id,
                        'term': term_value[2:]  # Remove 'P:' prefix
                    })
                elif term_value.startswith('C:'):
                    categorized_terms['cellular_component'].append({
                        'id': go_id,
                        'term': term_value[2:]  # Remove 'C:' prefix
                    })
        
        return {
            'categorized': categorized_terms  # New categorized format
}
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching GO annotations for {protein_id}: {e}")
        return {
            'categorized': {
                'molecular_function': [],
                'biological_process': [],
                'cellular_component': []
            }
        }
    '''

# STEP 1 

 # Define the UniProt API URL for XML data
def fetch_go_annotations(protein_id):
    """
    Fetch GO annotations and create GO ID to protein list mapping.
    
    Args:
        protein_ids (list): List of UniProt protein IDs
        
    Returns:
        List : List of the GO ids found for that protein
    """
    go_ids = []
    

    url = f"https://rest.uniprot.org/uniprotkb/{protein_id}.xml"

    try:
        response = requests.get(url)
        response.raise_for_status()
        
        namespaces = {'ns': 'http://uniprot.org/uniprot'}
        root = ET.fromstring(response.content)
        
        for db_ref in root.findall(".//ns:dbReference[@type='GO']", namespaces):
            go_id = db_ref.attrib.get('id')
            
            if go_id:
                go_ids.append(go_id)
                
    except requests.exceptions.RequestException as e:
        print(f"Error fetching GO annotations for {protein_id}: {e}")
   
            
    return go_ids



def fetch_go_terms(protein_ids):

    go_terms = {}

    for protein_id in protein_ids:
        url = f"https://rest.uniprot.org/uniprotkb/{protein_id}.xml"
        
        try:
            response = requests.get(url)
            response.raise_for_status()
            
            namespaces = {'ns': 'http://uniprot.org/uniprot'}
            root = ET.fromstring(response.content)
            
            for db_ref in root.findall(".//ns:dbReference[@type='GO']", namespaces):
                go_id = db_ref.attrib.get('id')
                term = db_ref.find("ns:property[@type='term']", namespaces)
                if go_id and term is not None:
                    go_term = term.get('value')
                    go_terms[go_id] = go_term
                    
        except requests.exceptions.RequestException as e:
            print(f"Error fetching GO terms for {protein_id}: {e}")
                
    return go_terms

# Let's add some debugging to help understand what's happening
# here we see that the big .xml file has the same structure as the small ones 
# we already analyzed ; thus,we can use the same parsing structure, but this time directly
# just collect the counts of GO terms, because that is all we need (no diff. categories, would just make our code slower)
def print_swissprot_file(swissprot_xml_path, length = 50):
    """
    Just to look at the first few lines to see the structure
    """

    with open(swissprot_xml_path, 'r') as f:
        print("First length lines of the file:")
        for i, line in enumerate(f):
            if i < length:
                print(line.strip())
            else:
                break



'''
def parse_swissprot_go_terms(swissprot_xml_path, family_proteins, skip_proteins):
    """
    Parse GO terms from SwissProt XML file, excluding proteins from our family.
    
    Args:
        swissprot_xml_path (str): Path to the SwissProt XML file
        family_proteins (set): Set of UniProt IDs in our protein family
        skip_proteins (bool): Whether to skip proteins in our family
    
    Returns:
        tuple: (go_term_counts dictionary, total proteins processed)
    """
    # Initialize counters
    go_term_counts = defaultdict(int)
    total_proteins = 0
    skipped_proteins = 0
    
    # Set up namespace for XML parsing
    namespaces = {'ns': 'http://uniprot.org/uniprot'}
    
    # Use iterparse for memory-efficient parsing
    context = ET.iterparse(swissprot_xml_path, events=('end',))
    
    print("Starting to parse SwissProt XML...")
    
    for event, elem in context:
        if elem.tag.endswith('entry'):
            # Get the UniProt ID for this protein
            accession = elem.find(".//ns:accession", namespaces)
            if accession is not None:
                uniprot_id = accession.text
                
                # Skip if this protein is in our family (we need this for the enrichment task to create the contigency table later on)
    
                if uniprot_id in family_proteins and skip_proteins:
                        skipped_proteins += 1
                else:
                    # Process GO terms for non-family proteins
                    for db_ref in elem.findall(".//ns:dbReference[@type='GO']", namespaces):
                        go_id = db_ref.attrib.get('id')
                        if go_id:
                            go_term_counts[go_id] += 1
                    total_proteins += 1
        

            
            # Clear the element to save memory
            elem.clear()
            
            # Print progress periodically
            if (total_proteins + skipped_proteins) % 10000 == 0:
                print(f"Processed {total_proteins} proteins "
                      f"(skipped {skipped_proteins} family proteins)...")
              #  break
        

    
    return go_term_counts, total_proteins
    '''

def parse_swissprot_go_terms(swissprot_xml_path, family_proteins):
   """
   Parse GO terms from SwissProt XML file for each protein.
   
   Args:
       swissprot_xml_path (str): Path to SwissProt XML file
       family_proteins (set): UniProt IDs in protein family
   
   Returns:
       dict: protein ID -> list of GO IDs for that protein
   """
   protein_to_go = defaultdict(list)
   total_proteins = 0
   skipped_proteins = 0
   
   namespaces = {'ns': 'http://uniprot.org/uniprot'}
   context = ET.iterparse(swissprot_xml_path, events=('end',))
   
   print("Starting to parse SwissProt XML...")
   
   for event, elem in context:
       if elem.tag.endswith('entry'):
           accession = elem.find(".//ns:accession", namespaces)
           if accession is not None:
               uniprot_id = accession.text
               
               if uniprot_id in family_proteins:
                   skipped_proteins += 1
               else:
                   for db_ref in elem.findall(".//ns:dbReference[@type='GO']", namespaces):
                       go_id = db_ref.attrib.get('id')
                       if go_id:
                           protein_to_go[uniprot_id].append(go_id)
                   total_proteins += 1

           elem.clear()
           
           if (total_proteins + skipped_proteins) % 10000 == 0:
               print(f"Processed {total_proteins} proteins "
                     f"(skipped {skipped_proteins} family proteins)...")
               
                    
               
   return protein_to_go

def calculate_go_enrichment(go_to_proteins_family, go_to_proteins_swissprot, total_proteins_family, total_proteins_swissprot, go_id_to_go_term):
    results = []
    
    
    for go_id in go_to_proteins_family.keys():
   
        # Create the 2x2 contingency table for Fisher's exact test
        # The table looks like this:
        #                   Protein in family    Protein not in family (i.e. all in SwissProt - family proteins)
        # Has GO term            a                    b
        # No GO term             c                    d
        
        # Contingency table calculations:
        a = len(go_to_proteins_family[go_id])  # Proteins with this GO term in family
        
        # For b, we need to make sure we don't subtract more than what's in SwissProt
        b = len(go_to_proteins_swissprot.get(go_id, []))  # Proteins with GO term in rest of SwissProt 
        
        c = total_proteins_family - a  # Proteins without GO term in family
        
        # For d, ensure we don't get negative values by using max
        d = total_proteins_swissprot - b
        
        # Verify all values are non-negative before creating contingency table
        if all(x >= 0 for x in [a, b, c, d]):
            contingency_table = [[a, b], [c, d]]
            
            # Perform Fisher's exact test
            # We ask : is the GO term appearing more often in our family than we would expect by random chance ?
            # The null hypothesis (H0) is: "The proportion of proteins with this GO term in our family 
            # is the same as the proportion in the SwissProt dataset (without the protein in the family)." 
            # In other words, under H0, getting the GO term is independent of being in our family (so it doesn't represent the family)
            # Alternative Hypothesis (H1) depends on what tail to use 
            #Right-tail (greater): Our family has a higher proportion of this GO term than SwissProt
            #Left-tail (less): Our family has a lower proportion of this GO term than SwissProt
            #Two-tail (two-sided): The proportion is different (either higher or lower)
            #Fisher's exact test calculates the probability of seeing our observed data (or more extreme) under the null hypothesis.
            #A very small p-value (like < 0.05) tells us:
            #Two-tail: This GO term's frequency is significantly different from SwissProt
            #Right-tail: This GO term is significantly enriched in our family(overrepresented)
            #Left-tail: This GO term is significantly depleted in our family(underrepresented)

            odds_ratio, pvalue_two_tail = fisher_exact(contingency_table, alternative='two-sided')
            # TODO : including both the p-values for now, we have to understand when to use what (like asked in the task), 
            # TODO : i.e. how we ordered the confusion matrix (contingency table)
            _, pvalue_greater = fisher_exact(contingency_table, alternative='greater')
          #  _, pvalue_less = fisher_exact(contingency_table, alternative='less')
            
            # Calculate fold enrichment safely
            my_proportion = a / total_proteins_family 
            swissprot_proportion = (a+b) / (total_proteins_swissprot + total_proteins_family)
     
            # Not needed anymore when we do a+b for the swissprot proportion : So we calculate contingency table and Fishers Test
            # on the right values now, but the proportion on ALL swissprot, so also the ones that are in family TODO : ask prof if correct
            '''
            # Fold Enrichment
            # TODO : see if the argumentation in the next comment makes sense (send email to prof)
            if b == 0: # When the swissprot count is 0, it means that : 
                                     # When collecting the GO terms of SwissProt, we skipped over the proteins in our family
                                     # Thus, if no protein in SwissProt has this GO term, ONLY the protein in the family itself 
                                     # has that GO term (compared to ALL of SwissProt), thus in the WordCloud later on
                                     # we want to especially show the term of this GO id and will thus give it
                                     # 'inf' amount (infinite) for now
                if my_proportion > 0:
                    fold_enrichment = float('inf')
                else:
                    fold_enrichment = 0
            else:
                fold_enrichment = my_proportion/swissprot_proportion
            '''
       
     
            
            results.append({
                'GO_ID': go_id,
                'GO_Term': go_id_to_go_term.get(go_id, 'N/A'),
                'Count_Prot_Dataset': a,
                'Count_Prot_SwissProt': b,
                'Count_Prot_SwissProt_Actual': a+b,
                'Percentage_Dataset': round(my_proportion * 100, 2),
                'Percentage_SwissProt': round(swissprot_proportion * 100, 10),
                'Fold_Enrichment': round(my_proportion/swissprot_proportion,2),
                'P_Value_Two_Tail': pvalue_two_tail,
                'P_Value_Greater': pvalue_greater,
            })
    
    # Convert to DataFrame and sort by p-value
    df_results = pd.DataFrame(results)
    if not df_results.empty:
        df_results = df_results.sort_values('P_Value_Two_Tail')

    df_results.to_csv("enrichment_results.csv")
    
    return df_results

In [ ]:
# Hierarchical Structure
def analyze_go_hierarchy():
    # First, we downloaded the go.obo file so we can parse it 
    go_obo = obo_parser.GODag('go.obo')
    
    # Read our enrichment results
    df = pd.read_csv("enrichment_results.csv")
    
    # Filter for significantly enriched terms
    enriched_terms = df[
        (df['P_Value_Two_Tail'] < 0.05) &
        (df['P_Value_Greater'] < 0.05)
    ]
    
    # Create a dictionary to store branch information
    branch_info = {}
    
    # For each enriched term, traverse up its ancestry
    for _, row in enriched_terms.iterrows():
        go_id = row['GO_ID']
        if go_id in go_obo:
            term = go_obo[go_id]
            
            # Get all ancestors (parents) up to the root of the DAG (since we use get_all_parents we do that here! get_parents would just get the direct parents)
            ancestors = term.get_all_parents()
            
            # Add information about this term to all its ancestor branches
            for ancestor_id in ancestors:
                if ancestor_id not in branch_info:
                    branch_info[ancestor_id] = {
                        'term_name': go_obo[ancestor_id].name,
                        'enriched_children': [],
                        'total_significance': 0,
                        'depth': go_obo[ancestor_id].depth,
                    }

                # TODO : correct ????
                # Our go_id is a child to the current ancestors (note that this is not necessarily a direct child, but maybe also much more down in the tree somewhere)
                branch_info[ancestor_id]['enriched_children'].append({
                    'id': go_id,
                    'name': term.name,
                    'p_value': row['P_Value_Two_Tail']
                })
                # Measure significance based on -log value of the p value of all the childs of the ancestor (lower p values have higher -log scores!)
                branch_info[ancestor_id]['total_significance'] += -np.log10(row['P_Value_Two_Tail'])
    
    # Filter for high-level terms (lower depth) with multiple enriched children
    significant_branches = {
        go_id: info for go_id, info in branch_info.items() # take each key,value of the branch_info dictionary
        if len(info['enriched_children']) >= 2  # At least 2 enriched children
        and info['depth'] <= 3  # High-level term (adjust this threshold as needed)
    }
    
    # Sort branches by their total significance
    sorted_branches = sorted(
        significant_branches.items(),
        key=lambda x: x[1]['total_significance'],
        reverse=True
    )
    
    # Create a list to store the branch information
    branch_data = []

    # Convert the branch information into a format suitable for a DataFrame
    for go_id, info in sorted_branches[:20]:  # Top 20 branches
        branch_data.append({
            'GO_ID': go_id,
            'Branch_Name': info['term_name'],
            'Hierarchy_Depth': info['depth'],
            'Number_Enriched_Terms': len(info['enriched_children']),
            'Total_Significance_Score': info['total_significance']
        })

    # Create a DataFrame and save to CSV
    branches_df = pd.DataFrame(branch_data)
    branches_df.to_csv('enriched_branches.csv', index=False)

In [ ]:
def main():

    psiblast_file = "psiblast_parsed.csv"
    hmm_file = "hmmsearch_output.csv"
    protein_ids = load_protein_ids(psiblast_file, hmm_file)
    

    
    # Proteins_to_GO terms for our family 
    print("Fetching GO annotations...")
    family_annotations = {}
    for pid in tqdm(protein_ids, desc="Fetching GO annotations"):
        family_annotations[pid] = fetch_go_annotations(pid)

    total_proteins_family = len(family_annotations)

    go_id_to_go_term = fetch_go_terms(protein_ids)

    
    # Proteins_to_GO terms for SwissProt
    swissprot_annotations = parse_swissprot_go_terms("uniprot_sprot.xml", protein_ids) #go_counts_swissprot, num_proteins_swissprot
    
    total_proteins_swissprot = len(swissprot_annotations)

    # Now Map the GO terms to the proteins ; for the enrichment task, we need to know how many proteins have a certain GO term
    go_to_proteins_swissprot = reverse_protein_go_dict(swissprot_annotations)
    go_to_proteins_family = reverse_protein_go_dict(family_annotations)
   



    
    # Calculate GO enrichments for both with skipped proteins and without 
    _ = calculate_go_enrichment(go_to_proteins_family, go_to_proteins_swissprot, total_proteins_family, total_proteins_swissprot, go_id_to_go_term)
    
    
    
    # Read the enrichment results
    df = pd.read_csv("enrichment_results.csv")

    # Get the terms to the GO ids from the family data
  #  go_id_to_term = create_go_id_to_term_mapping(family_annotations)

    # Filter for significantly enriched terms
    enriched_terms = df[
    (df['P_Value_Two_Tail'] < 0.05) &
    (df['P_Value_Greater'] < 0.05)
    ]


    # Create word frequencies using the actual GO terms instead of IDs
    word_frequencies = {}
    for _, row in enriched_terms.iterrows():
        go_id = row['GO_ID']
        if go_id in go_id_to_go_term:  # Make sure we have the term for this ID
            term = go_id_to_go_term[go_id]
            # Use fold enrichment as weight
            weight = row['Fold_Enrichment']
            word_frequencies[term] = weight

    # Create and display the word cloud
    wordcloud = WordCloud(
        width=1200, 
        height=800,
        background_color='white',
        prefer_horizontal=0.7,
        max_words=50,  # Limit to top 50 terms for better readability
        min_font_size=10,
        max_font_size=60
    ).generate_from_frequencies(word_frequencies)

    # Plot and save the word cloud
    plt.figure(figsize=(20, 12))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title('GO Term Enrichment Word Cloud', fontsize=16, pad=20)
    plt.savefig('go_enrichment_wordcloud.png', dpi=300, bbox_inches='tight')
    plt.close()

    # Print out the enriched terms for verification
    print("\nTop enriched GO terms:")
    sorted_terms = sorted(word_frequencies.items(), key=lambda x: x[1], reverse=True)
    for term, weight in sorted_terms[:10]:
        print(f"\nTerm: {term}")
        print(f"Weight in word cloud: {weight:.2f}")

    
    # Hierarchy

    analyze_go_hierarchy()

    
if __name__ == "__main__":
    main()

### Motifs
1. Search significantly conserved short motifs inside your family. Use ELM classes and ProSite patterns (for ProSite consider only patterns “PA” lines, not the profiles). Make sure to consider as true matches only those that are found inside disordered regions. Disordered regions for the entire SwissProt (as defined by MobiDB-lite) are available here